In [1]:
import pandas as pd 
import scipy.stats as st
import numpy as np


# Mostrar todo el contenido de las celdas (sin recortes)
pd.set_option('display.max_colwidth', None)

# Mostrar todas las columnas 
pd.set_option('display.max_columns', None)

In [2]:
ruta = "/home/donsson/proyectos/MODELO ABASTECIMIENTO/csvsalidas/clasificacion_mia2025.csv"

dp = pd.read_csv(ruta)

In [3]:
dp = dp.drop(columns=["Unnamed: 0"])

dp["store_name"] = dp["store_name"].replace({"PRINCIPAL COTA":"MOSTRADOR COTA"})

In [4]:
stock_seguridad = {
    "SUCURSAL CALLE 6": 50.00,
    "SUCURSAL VALLADOLID": 50.00,
    "SUCURSAL NORTE": 50.00,
    "SUCURSAL MEDELLIN": 150.00,
    "MOSTRADOR COTA": 300.00,
    "SUCURSAL BARRANQUILLA": 150.00,
    "SUCURSAL BUCARAMANGA": 150.00,
    "SUCURSAL CALI": 150.00,
   # "ANTIOQUEÑA DE LUBRICANTES SGP SAS": 300.00
}

reorden_ideal = {
    "SUCURSAL CALLE 6": {"Punto de reorden": 4.00, "Inventario ideal": 5.00},
    "SUCURSAL VALLADOLID": {"Punto de reorden": 4.00, "Inventario ideal": 5.00},
    "SUCURSAL NORTE": {"Punto de reorden": 4.00, "Inventario ideal": 5.00},
    "SUCURSAL MEDELLIN": {"Punto de reorden": 7.00, "Inventario ideal": 9.00},
    #"ANTIOQUEÑA DE LUBRICANTES SGP SAS": {"Punto de reorden": 5.00, "Inventario ideal": 7.00},
    "SUCURSAL BARRANQUILLA": {"Punto de reorden": 7.00, "Inventario ideal": 9.00},
    "SUCURSAL BUCARAMANGA": {"Punto de reorden": 7.00, "Inventario ideal": 9.00},
    "MOSTRADOR COTA": {"Punto de reorden": 2.50, "Inventario ideal": 3.00},
    "SUCURSAL CALI": {"Punto de reorden": 7.00, "Inventario ideal": 9.00}, # Tiene para 7 semnaas de inventario , y cubre el inventario 9 semanas
}


In [5]:
dp.head(10)

,store_name,product_ref,Año,semana,Ema,Costo unitario,Venta costo EMA,Venta costo %,Acumulado costo,Desviacion EMA,Desviacion EMA %,acumulado_cantidad,coef_ventas,acumulado_desviacion,acumulado_desviacion_p,acumulado_combinado,Clasificacion,Nivel de servicio
0,MOSTRADOR COTA,DCS10536137,2025.0,31,24.297362,51663.09,1255276.80,0.0029,0.0029,1029327.0,0.82,0.00,8.854054,0.82,1.0,0.2506,AAA,0.999
1,MOSTRADOR COTA,BCE00606125,2025.0,32,24.029336,20537.78,493509.22,0.0011,0.0548,2067803.6,4.19,0.00,16.200000,5.01,1.0,0.2694,AAA,0.999
2,MOSTRADOR COTA,DCS10536137,2025.0,42,22.271082,51663.09,1150592.91,0.0026,0.0055,989509.9,0.86,0.01,8.295374,5.87,1.0,0.2607,AAA,0.999
3,MOSTRADOR COTA,DCS10536137,2025.0,32,21.867626,51663.09,1129749.13,0.0026,0.0081,858609.3,0.76,0.01,7.238298,6.63,1.0,0.2607,AAA,0.999
4,MOSTRADOR COTA,DCS10536137,2025.0,43,20.043974,51663.09,1035533.63,0.0024,0.0105,766294.9,0.74,0.01,7.227666,7.37,1.0,0.2607,AAA,0.999
5,MOSTRADOR COTA,DCS10536137,2025.0,16,19.738068,51663.09,1019729.58,0.0023,0.0128,662824.2,0.65,0.01,10.185185,8.02,1.0,0.2607,AAA,0.999
6,MOSTRADOR COTA,DCS10536137,2025.0,33,19.680863,51663.09,1016774.20,0.0023,0.0151,843922.6,0.83,0.01,7.506383,8.85,1.0,0.2644,AAA,0.999
7,MOSTRADOR COTA,DCS10536137,2025.0,28,19.390071,51663.09,1001750.98,0.0023,0.0174,901575.9,0.90,0.01,11.685484,9.75,1.0,0.2644,AAA,0.999
8,MOSTRADOR COTA,BCE00606125,2025.0,33,19.223469,20537.78,394807.38,0.0009,0.0690,398755.5,1.01,0.01,26.880000,10.76,1.0,0.2832,AAA,0.999
9,MOSTRADOR COTA,DCS10536137,2025.0,44,18.039577,51663.09,931980.29,0.0021,0.0195,717624.8,0.77,0.02,7.417867,11.53,1.0,0.2707,AAA,0.999


## INVENTARIO DE SEGURIDAD

In [6]:
# Copia del df original
df_std = dp.copy()

# Ordenar datos por tienda, producto, año y semana
df_std = df_std.sort_values(by=["store_name", "product_ref", "Año", "semana"])

# Calcular la desviación móvil (rolling) de EMA con ventana de 12 semanas
df_std["Desviacion_EMA_calc"] = (
    df_std.groupby(["store_name", "product_ref"])["Ema"]
    .transform(lambda x: x.rolling(window=42, min_periods=1).std())
)

# Calcular también la media móvil del EMA (opcional, para validación)
df_std["Ema_media"] = (
    df_std.groupby(["store_name", "product_ref"])["Ema"]
    .transform(lambda x: x.rolling(window=42, min_periods=1).mean())
)

# === INVENTARIO IDEAL y PUNTO DE REORDEN ===
# Diccionario con los dos parámetros (en semanas de cobertura)
reorden_ideal = {
    "SUCURSAL CALLE 6": {"Punto de reorden": 4.00, "Inventario ideal": 5.00},
    "SUCURSAL VALLADOLID": {"Punto de reorden": 4.00, "Inventario ideal": 5.00},
    "SUCURSAL NORTE": {"Punto de reorden": 4.00, "Inventario ideal": 5.00},
    "SUCURSAL MEDELLIN": {"Punto de reorden": 7.00, "Inventario ideal": 9.00},
    "SUCURSAL BARRANQUILLA": {"Punto de reorden": 7.00, "Inventario ideal": 9.00},
    "SUCURSAL BUCARAMANGA": {"Punto de reorden": 7.00, "Inventario ideal": 9.00},
    "MOSTRADOR COTA": {"Punto de reorden": 2.50, "Inventario ideal": 3.00},
    "SUCURSAL CALI": {"Punto de reorden": 7.00, "Inventario ideal": 9.00},
}

# Extraer columnas de reorden e ideal
df_std["lt_reorden"] = df_std["store_name"].map(lambda x: reorden_ideal.get(x, {}).get("Punto de reorden", np.nan))
df_std["lt_ideal"] = df_std["store_name"].map(lambda x: reorden_ideal.get(x, {}).get("Inventario ideal", np.nan))


# Calcular Z-score según nivel de servicio
df_std["Z"] = df_std["Nivel de servicio"].apply(lambda x: st.norm.ppf(x))

# === STOCK DE SEGURIDAD (rolling 12 semanas) ===
df_std["inventario_seguridad"] = (
    df_std["Z"] * df_std["Desviacion_EMA_calc"] * np.sqrt(df_std["lt_ideal"])
)

# Inventario ideal (nivel máximo recomendado)
df_std["inventario_ideal"] = (df_std["Ema"] * df_std["lt_ideal"]) + df_std["inventario_seguridad"]


# Punto de reorden (cuándo pedir)
df_std["punto_reorden"] = (df_std["Ema"] * df_std["lt_reorden"]) + df_std["inventario_seguridad"]



# === Redondear columnas numéricas ===
df_std["inventario_seguridad"] = df_std["inventario_seguridad"].round(2)
df_std["inventario_ideal"] = df_std["inventario_ideal"].round(2)
df_std["punto_reorden"] = df_std["punto_reorden"].round(2)




df_std = df_std.drop(columns=["Desviacion_EMA_calc","Ema_media","lt_reorden","lt_ideal","Z"])

# TARDA 2 MINUTOS 

In [7]:
df_std[(df_std["semana"]==46) & (df_std["store_name"]=="SUCURSAL BUCARAMANGA")].sort_values(by=["Ema"],ascending=False).head(10)

,store_name,product_ref,Año,semana,Ema,Costo unitario,Venta costo EMA,Venta costo %,Acumulado costo,Desviacion EMA,Desviacion EMA %,acumulado_cantidad,coef_ventas,acumulado_desviacion,acumulado_desviacion_p,acumulado_combinado,Clasificacion,Nivel de servicio,inventario_seguridad,inventario_ideal,punto_reorden
362855,SUCURSAL BUCARAMANGA,DAB02570025,2025.0,46,38.153796,14058.32,536378.27,0.0003,0.0703,134094.6,0.25,0.02,3.774172,2.69,1.0,0.3126,AAA,0.999,85.83,429.21,352.90
362870,SUCURSAL BUCARAMANGA,DAB14570025,2025.0,46,34.427480,10188.65,350769.54,0.0002,0.1597,350769.5,1.00,0.03,6.665025,6.33,1.0,0.3559,AAA,0.999,97.49,407.34,338.48
363013,SUCURSAL BUCARAMANGA,DAB02666025,2025.0,46,19.577064,11624.90,227581.41,0.0001,0.2703,61447.0,0.27,0.09,3.249280,41.91,1.0,0.4411,AAA,0.999,41.85,218.05,178.89
363021,SUCURSAL BUCARAMANGA,BCS00249125,2025.0,46,19.344252,27218.11,526513.98,0.0003,0.0748,94772.5,0.18,0.09,2.767218,43.83,0.8,0.3058,AAA,0.999,29.82,203.92,165.23
363063,SUCURSAL BUCARAMANGA,BCS00035125,2025.0,46,18.460793,20467.02,377837.42,0.0002,0.1439,75567.5,0.20,0.10,3.053793,51.91,1.0,0.3951,AAA,0.999,33.02,199.16,162.24
363085,SUCURSAL BUCARAMANGA,DAB02738025,2025.0,46,17.982647,7027.71,126376.83,0.0001,0.4696,37913.0,0.30,0.11,5.618076,56.63,1.0,0.5359,A,0.950,24.66,186.50,150.53
363105,SUCURSAL BUCARAMANGA,BCS00180125,2025.0,46,17.409202,40117.68,698416.79,0.0004,0.0310,125715.0,0.18,0.12,2.775385,60.94,0.8,0.3099,AAA,0.999,26.47,183.15,148.33
363120,SUCURSAL BUCARAMANGA,DCS10352138,2025.0,46,17.141080,86802.01,1487880.20,0.0008,0.0008,892728.1,0.60,0.12,9.338889,65.08,1.0,0.3518,AAA,0.999,30.02,184.29,150.01
363146,SUCURSAL BUCARAMANGA,DAB02547025,2025.0,46,16.626680,22970.28,381919.50,0.0002,0.1405,87841.5,0.23,0.13,4.017107,71.23,1.0,0.4154,AAA,0.999,33.45,183.09,149.84
363157,SUCURSAL BUCARAMANGA,BCS00242125,2025.0,46,16.405810,39788.01,652754.53,0.0004,0.0418,163188.6,0.25,0.13,3.504274,73.63,1.0,0.3748,AAA,0.999,33.39,181.04,148.23


In [8]:
df_std.to_csv("/home/donsson/proyectos/MODELO ABASTECIMIENTO/csvsalidas/despachos.csv")

# INVENTARIO ACTUAL

In [9]:
ruta = "/home/donsson/proyectos/INVENTARIO/inventario_limpio.xlsx"

inv = pd.read_excel(ruta)

inv["product_ref"] = inv["Complete name"].str.extract(r"\[([A-Z0-9]+)\]")

In [10]:
inv.head(10)

,Sucursal,Estado,Codigo_interno,Complete name,Clase,Marca,categoria_producto,Cantidad,Costo unitario,Costo total,product_ref
0,PRINCIPAL COTA,Existencias,004 DA4207,[DAE04207004] DA4207 FILTRO AIRE SEGURIDAD KOMATSU (004 AF574SY),C,AUT*PARTS,Filtro,1.0,1.00,1.00,DAE04207004
1,PRINCIPAL COTA,Existencias,004 G314,[DLE00314004] G314 FILTRO ACEITE ESPECIAL (004 F62H),C,AUT*PARTS,Filtro,2.0,1890.00,3780.00,DLE00314004
2,PRINCIPAL COTA,Existencias,004 G401,[DLE00401004] G401 FILTRO ACEITE FIAT-ALLIS 75215486 (004 F75215486),C,AUT*PARTS,Filtro,7.0,1.00,7.00,DLE00401004
3,SUCURSAL MEDELLIN,Existencias,004 GA128,[DAE05128004] GA128 FILTRO AIRE FIAT DATSUN (004 GA128),C,AUT*PARTS,Filtro,2.0,1694.96,3389.92,DAE05128004
4,PRINCIPAL COTA,Existencias,004 GA128,[DAE05128004] GA128 FILTRO AIRE FIAT DATSUN (004 GA128),C,AUT*PARTS,Filtro,1.0,1694.96,1694.96,DAE05128004
5,PRINCIPAL COTA,Existencias,004 GA158,[DAE05158004] GA158 FILTRO AIRE RENULT 4 Y 6. (004 GA158),C,AUT*PARTS,Filtro,1.0,1722.50,1722.50,DAE05158004
6,PRINCIPAL COTA,Existencias,004 GA182,[DAE05182004] GA182 FILTRO AIRE SIMCA 1000 Y 1300 (004 GA182),C,AUT*PARTS,Filtro,1.0,1462.50,1462.50,DAE05182004
7,PRINCIPAL COTA,Existencias,004 GA321,[DAE05321004] GA321 FILTRO AIRE MONZA (004 GA321),C,AUT*PARTS,Filtro,2.0,1716.00,3432.00,DAE05321004
8,SUCURSAL BUCARAMANGA,Existencias,004 GS050,"[DLS00050004] GS050 FILTRO ACEITE SWIFT,SPRINT. (004 GS050)",C,AUT*PARTS,Filtro,1.0,6293.12,6293.12,DLS00050004
9,SUCURSAL NORTE,Existencias,004 GS050,"[DLS00050004] GS050 FILTRO ACEITE SWIFT,SPRINT. (004 GS050)",C,AUT*PARTS,Filtro,1.0,6293.12,6293.12,DLS00050004


In [11]:
inv["Cantidad"].sum() 

np.float64(316217.89300000004)

# EVALUACION Y COMPARACION

In [12]:
import xmlrpc.client
import pandas as pd


# ===============================
# Conexión con Odoo
# ===============================
username = "juan.cano@donsson.com"   # tu usuario
password = "1000285668"              # tu contraseña
url = "https://donsson.com"          # URL del servidor
db = "Donsson_produccion"            # nombre de la base de datos

common = xmlrpc.client.ServerProxy(f"{url}/xmlrpc/2/common")
uid = common.authenticate(db, username, password, {})
models = xmlrpc.client.ServerProxy(f"{url}/xmlrpc/2/object")


#COSTO DE LOS PRODUCTOS

productos_ema = models.execute_kw(
    db, uid, password,
    "cs.analisis.costo", "search_read",[],
    #[
    #    [["semana","=",36]]
    #],
    {"fields": ["sucursal_id","product_name","ano","semana","ema","producto_costo_unitario","punto_reorden","clasificacion","almacen_id","inventario_seguridad","inventario_ideal" ]}
)

df = pd.DataFrame(productos_ema)

def extract_id(val):
    if isinstance(val, (list, tuple)) and len(val) > 0:
        return val[0]
    return None

def extract_name(val):
    if isinstance(val, (list, tuple)) and len(val) > 1:
        return val[1]
    return str(val)

df.rename(columns={"product_name": "product_ref"}, inplace=True)


df["store_id_num"] = df["sucursal_id"].apply(extract_id)
df["store_name"]   = df["sucursal_id"].apply(extract_name)

df["almacen_id_num"] = df["almacen_id"].apply(extract_id)
df["almacen_name"]   = df["almacen_id"].apply(extract_name)


In [13]:
df_41 = df_std[df_std["semana"]==46]

df_41["Clasificacion_mia"] = df_41["Clasificacion"]

/tmp/ipykernel_8243/2512265855.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_41["Clasificacion_mia"] = df_41["Clasificacion"]


In [14]:
# --- MERGE LIMPIO PARA COMPARAR ---
df_merge_simple = pd.merge(
    df_41[['store_name', 'product_ref', 'Clasificacion_mia', 
            'inventario_seguridad', 'inventario_ideal', 'punto_reorden']],
    df[['store_name', 'product_ref', 'clasificacion', 
         'inventario_seguridad', 'inventario_ideal', 'punto_reorden']],
    on=['store_name', 'product_ref'],
    how='inner',
    suffixes=('_std', '_real')
)

# --- COMPARACIÓN DE CLASIFICACIÓN ---
df_merge_simple['match_clasificacion'] = (
    df_merge_simple['Clasificacion_mia'] == df_merge_simple['clasificacion']
)

# --- OPCIONAL: ORDENAR PARA REVISAR FÁCIL ---
df_merge_simple = df_merge_simple.sort_values(by=['store_name', 'product_ref'])

# --- MOSTRAR SOLO LAS COLUMNAS RELEVANTES ---
cols = [
    'store_name', 'product_ref',
    'Clasificacion_mia', 'clasificacion', 'match_clasificacion',
    'inventario_seguridad_std', 'inventario_seguridad_real',
    'inventario_ideal_std', 'inventario_ideal_real',
    'punto_reorden_std', 'punto_reorden_real'
]

df_merge_simple = df_merge_simple[cols]



df_merge_simple.to_excel("/home/donsson/proyectos/MODELO ABASTECIMIENTO/csvsalidas/diferencias_ema.xlsx")

In [15]:
df_merge_simple.sample(10)

,store_name,product_ref,Clasificacion_mia,clasificacion,match_clasificacion,inventario_seguridad_std,inventario_seguridad_real,inventario_ideal_std,inventario_ideal_real,punto_reorden_std,punto_reorden_real
10961,SUCURSAL CALI,DAE01126136,C,C,True,0.00,0.000000,0.00,1,0.00,0
16881,SUCURSAL MEDELLIN,BAB02012125,C,C,True,0.00,0.000000,0.00,0,0.00,0
1315,SUCURSAL BARRANQUILLA,BHS00101125,C,C,True,0.00,0.000000,0.85,1,0.66,1
17294,SUCURSAL MEDELLIN,BAC00169125,C,C,True,0.00,0.000000,2.07,0,1.61,0
20273,SUCURSAL MEDELLIN,BLS00655125,C,C,True,0.00,0.000000,0.00,0,0.00,0
17218,SUCURSAL MEDELLIN,BAC00091125,C,C,True,0.00,0.000000,0.00,0,0.00,0
14207,SUCURSAL CALLE 6,BLS00205125,B,A,False,1.89,2.889207,11.35,13,9.46,11
11329,SUCURSAL CALI,DAE06834122,C,C,True,0.00,0.000000,0.00,0,0.00,0
28503,SUCURSAL NORTE,DAS00719189,C,C,True,0.00,0.000000,0.00,0,0.00,0
28788,SUCURSAL NORTE,DCE00906189,C,B,False,0.00,0.951540,4.38,6,3.50,5
